In [84]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sksparse.cholmod import cholesky as chol
from numpy.linalg import lstsq, cond
from scipy.optimize import broyden1

plt.close('all')

p1 = np.r_[0,0]
p2 = np.r_[1,0]
# p3 = np.r_[1/2,np.sqrt(3)/2]
p3 = np.r_[0,1]

m1 = (p2+p3)/2
m2 = (p1+p3)/2
m3 = (p1+p2)/2

b = (p1+p2+p3)/3

# vertices
T1 = np.c_[p1,p2,p3]
eval_T1 = lambda i,j : T1[0,:]**i*T1[1,:]**j

# edge midpoints
T2 = np.c_[m1,m2,m3]
eval_T2 = lambda i,j : T2[0,:]**i*T2[1,:]**j

# midpoint
T3 = np.c_[b]
eval_T3 = lambda i,j : T3[0,:]**i*T3[1,:]**j

# edge class
T4 = lambda a : np.c_[a*p1 + (1-a)*p2,
                      a*p2 + (1-a)*p1,
                      a*p3 + (1-a)*p1,
                      a*p1 + (1-a)*p3,
                      a*p3 + (1-a)*p2,
                      a*p2 + (1-a)*p3]

eval_T4 = lambda a,i,j : T4(a)[0,:]**(0*int(i<0)+i*int(i>=0))*T4(a)[1,:]**(0*int(j<0)+j*int(j>=0))

dT4 = np.c_[p1-p2,
            p2-p1,
            p3-p1,
            p1-p3,
            p3-p2,
            p2-p3]

eval_dT4 = lambda i,j : dT4[0,:]**(0*int(i<0)+i*int(i>=0))*dT4[1,:]**(0*int(j<0)+j*int(j>=0))

# inner class, type 1, TODO
T5 = lambda a : np.c_[a*m1 + (1-a)*p1,
                      a*m2 + (1-a)*p2,
                      a*m3 + (1-a)*p3]

dT5 = np.c_[m1-p1,
            m2-p2,
            m3-p3]

eval_T5 = lambda a,i,j : T5(a)[0,:]**(0*int(i<0)+i*int(i>=0))*T5(a)[1,:]**(0*int(j<0)+j*int(j>=0))

# inner class, type 2, TODO
T6 = lambda a,b : np.c_[b*(a*m1 + (1-a)*p1) + (1-b)*(a*m2 + (1-a)*p2),
                        b*(a*m1 + (1-a)*p1) + (1-b)*(a*m3 + (1-a)*p3),
                        b*(a*m3 + (1-a)*p3) + (1-b)*(a*m2 + (1-a)*p2),
                        b*(a*m2 + (1-a)*p2) + (1-b)*(a*m1 + (1-a)*p1),
                        b*(a*m3 + (1-a)*p3) + (1-b)*(a*m1 + (1-a)*p1),
                        b*(a*m2 + (1-a)*p2) + (1-b)*(a*m3 + (1-a)*p3)]

##### Evaluation #####

def integral(i,j):
    return math.factorial(j)*math.factorial(i)/math.factorial(i+j+2)

###### System ######

# order = 4
# Aij = lambda i,j,a: np.r_[sum(eval_T1(i,j)),
#                           sum(eval_T2(i,j)),
#                           sum(eval_T5(a,i,j))]

# order = 6
Aij = lambda i,j,a: np.r_[sum(eval_T1(i,j)),
                          sum(eval_T2(i,j)),
                          eval_T3(i,j),
                          sum(eval_T4(a[0],i,j)),
                          sum(eval_T5(a[1],i,j)),
                          sum(eval_T5(a[2],i,j))]


# Aij = lambda i,j,a: np.r_[eval_T1(i,j),
#                           eval_T2(i,j),
#                           eval_T5(a,i,j)]

A = lambda a: np.c_[Aij(0,0,a),
                    
                    Aij(1,0,a),
                    Aij(0,1,a),

                    Aij(1,1,a),
                    Aij(2,0,a),
                    Aij(0,2,a),

                    Aij(2,1,a),
                    Aij(1,2,a),
                    Aij(3,0,a),
                    Aij(0,3,a),

                    Aij(2,2,a),
                    Aij(1,3,a),
                    Aij(3,1,a),
                    Aij(4,0,a),
                    Aij(0,4,a),

                    Aij(1,4,a),
                    Aij(4,1,a),
                    Aij(2,3,a),
                    Aij(3,2,a),
                    Aij(0,5,a),
                    Aij(5,0,a),

                    Aij(1,5,a),
                    Aij(5,1,a),
                    Aij(2,4,a),
                    Aij(4,2,a),
                    Aij(3,3,a),
                    Aij(0,6,a),
                    Aij(6,0,a),
                    ]

rhs = np.r_[integral(0,0),
            
            integral(1,0),
            integral(0,1),

            integral(1,1),
            integral(2,0),
            integral(0,2),

            integral(2,1),
            integral(1,2),
            integral(3,0),
            integral(0,3),

            integral(2,2),
            integral(1,3),
            integral(3,1),
            integral(4,0),
            integral(0,4),

            integral(1,4),
            integral(4,1),
            integral(2,3),
            integral(3,2),
            integral(0,5),
            integral(5,0),

            integral(1,5),
            integral(5,1),
            integral(2,4),
            integral(4,2),
            integral(3,3),
            integral(0,6),
            integral(6,0),
            ]

In [85]:
a = [0.1,0.2,0.2]
np.linalg.norm(A(a).T@lstsq(A(a)@A(a).T,A(a)@rhs, rcond=-1)[0]-rhs)

7.175694044396493e-05

In [86]:
from numpy.linalg import solve,lstsq,cond
from scipy.optimize import broyden1,broyden2,diagbroyden,newton_krylov

f = lambda a,b,c : np.linalg.norm(A(a,b,c).T@lstsq(A(a,b,c)@A(a,b,c).T,A(a,b,c)@rhs, rcond=-1)[0]-rhs)



# f = lambda a : np.linalg.norm(A(a).T@solve(A(a)@A(a).T,A(a)@rhs)-rhs)

# print(f(0.3,0.2,0.5))

def f_wrapper(args): return f(*(args))
print(f_wrapper(np.array([0.7,0.3,0.4])))

# stop

num_a = broyden1(f_wrapper, [0.1,0.1,0.1], f_tol = 1e-12, line_search = 'armijo')

print("kek ", num_a)
print("Cond: ", cond(A(num_a)))

# aa = np.arange(0.2,0.5,0.001)

# import matplotlib.pyplot as plt

# faa = np.zeros(aa.size)
# for i,j in enumerate(aa):
#     faa[i] = f(j)

# plt.plot(aa,faa)

TypeError: <lambda>() takes 1 positional argument but 3 were given

In [63]:
# A(0,1,2)

In [ ]:
lam = 2*(7-np.sqrt(13))/18;
print(lam)

In [ ]:
# print(A(1).T)

f = lambda a : A(a).T@solve(A(a)@A(a).T,A(1)@rhs)

print(f(1))
print(rhs.T)

In [ ]:
import numpy as np
from numpy.linalg import lstsq, cond
from scipy.optimize import broyden1

# Assuming p1, p2, p3 are numpy arrays of the same length
p1 = np.array([1.0, 2.0, 3.0])
p2 = np.array([4.0, 5.0, 6.0])
p3 = np.array([7.0, 8.0, 9.0])

# Define your functions
def eval_T1(i, j):
    return np.array([i+j, i-j])

def eval_T3(i, j):
    return np.array([i*j, i/j]) if j != 0 else np.array([i*j, 0])

def eval_T4(a, i, j):
    T4 = lambda a: np.vstack([
        a*p1 + (1-a)*p2,
        a*p2 + (1-a)*p1,
        a*p3 + (1-a)*p1,
        a*p1 + (1-a)*p3,
        a*p3 + (1-a)*p2,
        a*p2 + (1-a)*p3
    ])
    T4_result = T4(a)
    return T4_result[0, :]**(0 if i < 0 else i) * T4_result[1, :]**(0 if j < 0 else j)

def eval_T5(a, i, j):
    return np.array([a + i, a - j])

def Aij(i, j, a):
    return np.concatenate((
        eval_T1(i, j),
        eval_T3(i, j),
        eval_T4(a[0], i, j),
        eval_T5(a[1], i, j),
        eval_T5(a[2], i, j)
    ))

def A(a):
    return np.column_stack((
        Aij(0, 0, a),
        Aij(1, 0, a),
        Aij(0, 1, a),
        Aij(1, 1, a),
        Aij(2, 0, a),
        Aij(0, 2, a),
        Aij(2, 1, a),
        Aij(1, 2, a),
        Aij(3, 0, a),
        Aij(0, 3, a),
        Aij(2, 2, a),
        Aij(1, 3, a),
        Aij(3, 1, a),
        Aij(4, 0, a),
        Aij(0, 4, a),
        Aij(1, 4, a),
        Aij(4, 1, a),
        Aij(2, 3, a),
        Aij(3, 2, a),
        Aij(0, 5, a),
        Aij(5, 0, a),
        Aij(1, 5, a),
        Aij(5, 1, a),
        Aij(2, 4, a),
        Aij(4, 2, a),
        Aij(3, 3, a),
        Aij(0, 6, a),
        Aij(6, 0, a)
    ))

# Adjust rhs to have the same number of elements as the number of columns in A(a)
rhs = np.arange(1, 29)  # Because A(a) has 28 columns

def f(a):
    A_a = A(a)
    print(f"A(a) shape: {A_a.shape}")
    print(f"A(a) @ A(a).T shape: {(A_a @ A_a.T).shape}")
    print(f"A(a) @ rhs shape: {(A_a @ rhs).shape}")

    if A_a.shape[1] != rhs.shape[0]:
        raise ValueError(f"Dimension mismatch: A(a).shape[1] = {A_a.shape[1]}, rhs.shape[0] = {rhs.shape[0]}")

    result = np.linalg.norm(A_a.T @ lstsq(A_a @ A_a.T, A_a @ rhs, rcond=None)[0] - rhs)
    return result

def f_wrapper(x):
    return f(np.array(x))

# Call broyden1 with the wrapper function
initial_guess = np.array([0.1, 0.1, 0.1])
num_a = broyden1(f_wrapper, initial_guess, f_tol=1e-15, line_search='armijo')

print("Result: ", num_a)
print("Condition Number: ", np.linalg.cond(A(num_a)))
